In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import torch.nn.functional as F
import torch
import sys
import os

# ===================================================================
# Step 1: Data Loading and Preparation
# ===================================================================
print("STEP 1: Loading and preparing data...")

# --- Ensure custom utils are in the path ---
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from utils.load_meso_session import MesoscopeSession
from one.api import ONE

# --- Load the real session data ---
one = ONE()
SESSION_INDEX = 0
session = MesoscopeSession.from_csv(one, '../good_mesoscope_sessions_final.csv', SESSION_INDEX, True)
activity_matrix, timestamps = session.get_activity_matrix()
print(f"Successfully loaded data. Activity matrix shape: {activity_matrix.shape}")


# --- Perform the same Z-score normalization as the Transformer model ---
mean_vals = np.mean(activity_matrix, axis=0)
std_vals = np.std(activity_matrix, axis=0)
epsilon = 1e-8
std_vals[std_vals < epsilon] = epsilon
activity_matrix_normalized = (activity_matrix - mean_vals) / std_vals
print("Normalization complete.")


# ===================================================================
# Step 2: Implement the 1-Step Autoregressive Baseline
# ===================================================================
print("\nSTEP 2: Applying the 1-step autoregressive model...")

# The prediction for time `t+1` is simply the value at time `t`.
# This is also known as a "persistence" or "naive" forecast.

# Our "predictions" are the entire time series, shifted by one step.
# We exclude the last time point because it has no future.
predictions = activity_matrix_normalized[:-1]

# The "ground truth" is the entire time series, starting from the second step.
ground_truth = activity_matrix_normalized[1:]

print(f"Baseline created. Comparing {len(predictions)} time steps.")


# ===================================================================
# Step 3: Calculate Performance Metrics
# ===================================================================
print("\nSTEP 3: Calculating baseline performance metrics...")

# --- A. Overall Performance (All Neurons, All Time) ---
# Convert to tensors for easy calculation with torch.nn.functional
gt_tensor = torch.from_numpy(ground_truth)
fc_tensor = torch.from_numpy(predictions)

overall_mse = F.mse_loss(fc_tensor, gt_tensor).item()
overall_mae = F.l1_loss(fc_tensor, gt_tensor).item()

print("\n--- Overall Baseline Performance (All Neurons, All Time) ---")
print(f"  Mean Squared Error (MSE): {overall_mse:.6f}")
print(f"  Mean Absolute Error (MAE): {overall_mae:.6f}")

# --- B. Per-Neuron Correlation ---
# How well does each neuron's activity at t+1 correlate with its activity at t?
correlations = []
for i in range(activity_matrix_normalized.shape[1]):
    # Use pearsonr; it returns (correlation, p-value)
    corr, _ = pearsonr(ground_truth[:, i], predictions[:, i])
    correlations.append(corr)
correlations = np.nan_to_num(correlations) # Replace NaNs (from flat lines) with 0

print(f"\n--- Per-Neuron Baseline Correlation ---")
print(f"  Mean Correlation across all {activity_matrix_normalized.shape[1]} neurons: {np.mean(correlations):.3f}")
print(f"  Median Correlation: {np.median(correlations):.3f}")
print(f"  Std. Dev. of Correlation: {np.std(correlations):.3f}")


# ===================================================================
# Step 4 (Optional): Extending to a 2-Second Future Window
# ===================================================================
print("\nSTEP 4: Evaluating for a 2-second future window (like the Transformer)...")

# --- Let's match the Transformer's prediction_length ---
# Assuming ~4.9 Hz, a 2-second prediction is about 10 steps
prediction_length = int(2 * 4.9)

# The baseline "forecast" for the next 10 steps is just the value at time `t`
# repeated 10 times.
num_windows = len(activity_matrix_normalized) - prediction_length
# Shape: (num_windows, prediction_length, num_neurons)
future_predictions_baseline = np.zeros((num_windows, prediction_length, activity_matrix_normalized.shape[1]))
future_ground_truth = np.zeros_like(future_predictions_baseline)

for i in range(num_windows):
    # The prediction for all future steps is the value at time `i`
    last_known_value = activity_matrix_normalized[i]
    future_predictions_baseline[i, :, :] = last_known_value
    
    # The actual future values
    future_ground_truth[i, :, :] = activity_matrix_normalized[i+1 : i+1+prediction_length]

# --- Calculate metrics for this multi-step forecast ---
gt_tensor_multi = torch.from_numpy(future_ground_truth)
fc_tensor_multi = torch.from_numpy(future_predictions_baseline)

multistep_mse = F.mse_loss(fc_tensor_multi, gt_tensor_multi).item()
multistep_mae = F.l1_loss(fc_tensor_multi, gt_tensor_multi).item()

print("\n--- Baseline Performance for 2-Second Forecast Window ---")
print(f"  Multi-step MSE: {multistep_mse:.6f}")
print(f"  Multi-step MAE: {multistep_mae:.6f}")

STEP 1: Loading and preparing data...


(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP066\2025-04-09\002\alf\FOV_00\mpci.ROIActivityF.npy: 100%|██████████| 231M/231M [00:26<00:00, 8.62MB/s] 
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP066\2025-04-09\002\alf\FOV_01\mpci.ROIActivityF.npy: 100%|██████████| 265M/265M [00:29<00:00, 8.88MB/s] 
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP066\2025-04-09\002\alf\FOV_02\mpci.ROIActivityF.npy: 100%|██████████| 343M/343M [00:36<00:00, 9.40MB/s] 
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP066\2025-04-09\002\alf\FOV_03\mpci.ROIActivityF.npy: 100%|██████████| 165M/165M [00:16<00:00, 10.3MB/s] 
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP066\2025-04-09\002\alf\FOV_04\mpci.ROIActivityF.npy: 100%|██████████| 235M/235M [00:25<00:00, 9.34MB/s] 
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationa

Successfully loaded data. Activity matrix shape: (19081, 7673)
Normalization complete.

STEP 2: Applying the 1-step autoregressive model...
Baseline created. Comparing 19080 time steps.

STEP 3: Calculating baseline performance metrics...

--- Overall Baseline Performance (All Neurons, All Time) ---
  Mean Squared Error (MSE): 0.835180
  Mean Absolute Error (MAE): 0.688976


C:\Users\Gugu\AppData\Local\Temp\ipykernel_248420\4008306189.py:78: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(ground_truth[:, i], predictions[:, i])



--- Per-Neuron Baseline Correlation ---
  Mean Correlation across all 7673 neurons: 0.582
  Median Correlation: 0.571
  Std. Dev. of Correlation: 0.161

STEP 4: Evaluating for a 2-second future window (like the Transformer)...


KeyboardInterrupt: 

In [3]:
future_predictions_baseline[0,0,:]

NameError: name 'future_predictions_baseline' is not defined